In [1]:
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_VISIBLE_DEVICES=2


In [2]:
from lightning_simclr import SimCLR
ckpt = "logs/No_schedule-_No_decay-1000/l2/no_decay-no_momentum/checkpoint/epoch=999.ckpt"
mod = SimCLR.load_from_checkpoint(ckpt, strict=False)

/gpfs01/berens/user/jdoehl/GitHub/sssl/.venv/lib/python3.12/site-packages/lightning/pytorch/core/saving.py:191: Found keys that are in the model state dict but not in the checkpoint: ['model.certainty_head.0.weight', 'model.certainty_head.0.bias', 'model.certainty_head.2.weight', 'model.certainty_head.2.bias']


In [4]:
mod

SimCLR(
  (model): ResNet18withProjector(
    (backbone): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): Identity()
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_r

In [ ]:
from lightning_simclr import SimCLR
from data_utils import get_augmented_dataloader
import lightning as L

dataset_name = "cifar10"

mod = SimCLR(dataset_name=dataset_name, n_epochs=1000, norm_reguralization_clip=0.00)
print(mod.hparams)
trainer = L.Trainer(max_epochs=20)
data_loader = get_augmented_dataloader(mod.dataset_name)
trainer.fit(mod, data_loader)

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


"base_lr":                  0.06
"batch_size":               1024
"cut":                      1
"dataset_name":             cifar10
"n_epochs":                 1000
"nestrov":                  False
"norm_function_str":        l2
"norm_reguralization_clip": 0.002
"power":                    0
"seed":                     1145799988
"warmup_epochs":            1
"weight_decay":             0.0005


You are using a CUDA device ('NVIDIA A40') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [2]

  | Name  | Type                  | Params | Mode 
--------------------------------------------------------
0 | model | ResNet18withProjector | 11.8 M | train
--------------------------------------------------------
11.8 M    Trainable params
0         Non-trainable params
11.8 M    Total params
47.301    Total estimated model params size (MB)
73        Modules in train mode
0         Modules in eval mode
/gpfs01/berens/user/jdoehl/GitHub/sssl/.venv/lib/python3.12/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (49) is smaller than the logging interval Trainer(l

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.


In [ ]:
import os
name = "epoch=999.ckpt"
log_dir = "logs/Unbalanced-1000"
fully_trained_ckpt_paths = []
for (root,dirs,files) in os.walk(log_dir, topdown=True):
    if name in files:
        fully_trained_ckpt_paths.append(root+"/"+name)
fully_trained_ckpt_paths

['logs/Unbalanced-1000/exp_map/version_2/checkpoint/epoch=999.ckpt',
 'logs/Unbalanced-1000/exp_map/version_0/checkpoint/epoch=999.ckpt',
 'logs/Unbalanced-1000/exp_map/version_1/checkpoint/epoch=999.ckpt',
 'logs/Unbalanced-1000/mono/version_2/checkpoint/epoch=999.ckpt',
 'logs/Unbalanced-1000/mono/version_0/checkpoint/epoch=999.ckpt',
 'logs/Unbalanced-1000/mono/version_1/checkpoint/epoch=999.ckpt',
 'logs/Unbalanced-1000/l2/version_2/checkpoint/epoch=999.ckpt',
 'logs/Unbalanced-1000/l2/version_3/checkpoint/epoch=999.ckpt',
 'logs/Unbalanced-1000/l2/version_5/checkpoint/epoch=999.ckpt',
 'logs/Unbalanced-1000/l2/version_7/checkpoint/epoch=999.ckpt',
 'logs/Unbalanced-1000/l2/version_4/checkpoint/epoch=999.ckpt',
 'logs/Unbalanced-1000/l2/version_0/checkpoint/epoch=999.ckpt',
 'logs/Unbalanced-1000/l2/version_6/checkpoint/epoch=999.ckpt',
 'logs/Unbalanced-1000/l2/version_1/checkpoint/epoch=999.ckpt',
 'logs/Unbalanced-1000/l2/version_8/checkpoint/epoch=999.ckpt',
 'logs/Unbalanced-1

In [ ]:
from lightning_simclr import SimCLR
from data_utils import get_train_and_test_set, CIFAR10, CIFAR10_Unbalanced
import os
name = "epoch=499.ckpt"
conidtion2 = "version_5"
log_dir = "logs/Unbalanced-1000"
specific_ckpts = []
for (root,dirs,files) in os.walk(log_dir, topdown=True):
    if name in files:
        if conidtion2 in root:
            specific_ckpts.append(root+"/"+name)
print(specific_ckpts)

def get_accs_for_ckpts(
        ckpt_paths,
        knn_datasets = {"cifar10_unbalanced": CIFAR10_Unbalanced}
        ):
    acc_dict = {key:{} for key in knn_datasets.keys()}
    for ckpt in ckpt_paths:
        mod = SimCLR.load_from_checkpoint(ckpt, strict=False)
        print(mod.norm_function_str)
        print("cut:", mod.hparams["cut"])
        for dataset_name, Dataset in knn_datasets.items():
            train, test = get_train_and_test_set(Dataset)
            acc = mod.get_knn_acc(train, test, n_neighbors=200)
            acc_dict[dataset_name] = acc
            print(f"{dataset_name}, knn acc:", acc)
        print()
    return acc_dict

acc_dict = get_accs_for_ckpts(specific_ckpts)

l2
cut: 1
cifar10_unbalanced, knn acc: 0.5837



In [ ]:
from lightning_simclr import SimCLR
from data_utils import get_train_and_test_set, CIFAR10, CIFAR10_Unbalanced
import os
name = "epoch=999.ckpt"
conidtion2 = "version_0"
log_dir = "logs/Unbalanced-1000"
specific_ckpts = []
for (root,dirs,files) in os.walk(log_dir, topdown=True):
    if name in files:
        if conidtion2 in root:
            specific_ckpts.append(root+"/"+name)


def get_accs_for_ckpts(
        ckpt_paths,
        knn_datasets = {"cifar10_unbalanced": CIFAR10_Unbalanced}
        ):
    acc_dict = {key:{} for key in knn_datasets.keys()}
    for ckpt in ckpt_paths:
        mod = SimCLR.load_from_checkpoint(ckpt, strict=False)
        print(mod.norm_function_str)
        print("cut:", mod.hparams["cut"])
        for dataset_name, Dataset in knn_datasets.items():
            train, test = get_train_and_test_set(Dataset)
            acc = mod.get_knn_acc(train, test, n_neighbors=200)
            acc_dict[dataset_name] = acc
            print(f"{dataset_name}, knn acc:", acc)
        print()
    return acc_dict

acc_dict = get_accs_for_ckpts(specific_ckpts)

exp_map
cut: 1
cifar10_unbalanced, knn acc: 0.5903

mono
cut: 1
cifar10_unbalanced, knn acc: 0.6137

l2
cut: 3
cifar10_unbalanced, knn acc: 0.6325

stereo
cut: 1
cifar10_unbalanced, knn acc: 0.6132



In [ ]:
import numpy as np


def mean_var(arr):
    print("mean:", np.mean(arr).item()*100, "std:", f"{np.std(arr).item()*100:.5f}")

# Unbalanced Cifar10
acc_500 = dict(
    l2 = [0.656, 0.662, 0.653],
    cut3 = [0.653, 0.637, 0.649],
    exp_map = [0.613, 0.621, 0.624],
    stereo = [0.631, 0.627, 0.629],
    mono = [0.639, 0.642, 0.637],
    # grad_scale = []
)

for key, value in acc_500.items():
    print(key)
    mean_var(value)
print()

acc_1k = dict(
    l2 = [0.689, 0.691, 0.693],
    cut3 = [0.69, 0.697, 0.695],
    exp_map = [0.648, 0.641, 0.639],
    stereo = [0.643, 0.642, 0.646],
    mono = [0.653, 0.656, 0.665],
    # grad_scale = []
)

for key, value in acc_1k.items():
    print(key)
    mean_var(value)

### Unbalanced Cifar10
A Cifar10 version where datapoints where systematically removed to make it unbalanced.  
For each variant three seeds were tested. The numbers shown in the table are: **Mean $\pm$ Std.**

| Epoch (out of 1000)     | L2   (Baseline)             | Cut                  | Exp Map              | Stereo            | Mono                 |
|------------------|-------------------|----------------------|----------------------|-------------------|----------------------|
| 500| **65.7 ± 0.37** | 64.3333 ± 0.67 | 61.9333 ± 0.46 | 62.9 ± 0.16 | 63.9333 ± 0.20 |  
| 1000 | 69.1 ± 0.16 | **69.4 ± 0.29** | 64.2666 ± 0.38 | 64.3666 ± 0.16 | 65.8 ± 0.5 |

In [ ]:
import numpy as np


def mean_var(arr):
    print("mean:", np.mean(arr).item()*100, "std:", f"{np.std(arr).item()*100:.5f}")

# Cifar10
acc_500 = dict(
    l2 = [0.87, 0.871, 0.869],
    cut3 = [0.868, 0.869, 0.865],
    exp_map = [0.854, 0.854, 0.855],
    stereo = [0.864, 0.862, 0.864],
    mono = [0.865, 0.866, 0.867],
    # grad_scale = []
)
print("epoch 500:")
for key, value in acc_500.items():
    print(key)
    mean_var(value)
print()

acc_1k = dict(
    l2 = [0.9018, 0.9047, 0.9026],
    cut3 = [0.901, 0.9045, 0.9036],
    exp_map = [0.8954, 0.8989, 0.8956],
    stereo = [0.9017, 0.9023, 0.902],
    mono = [0.9005, 0.9057, 0.9008],
    # grad_scale = []
)
print("epoch 1000:")
for key, value in acc_1k.items():
    print(key)
    mean_var(value)

### Cifar10
For each variant three seeds were tested. The numbers shown in the table are: **Mean ± Std.**

| Epoch (out of 1000)   | L2 (Baseline)  | Cut          | Exp Map          | Stereo        | Mono              |
|------------------|----------------|--------------|------------------|---------------|-------------------|
| 500  | **87 ± 0.08** | 86.73 ± 0.16 | 85.43 ± 0.04  | 86.33 ± 0.09 | 86.6 ± 0.08 | 
| 1000 | **90.30 ± 0.12** |**90.30 ± 0.14** | 89.66 ± 0.16  | 90.2 ± 0.24 | 90.23 ± 0.23 |